For the theoretical background regarding these techniques , pleases refer to theoretical discussion on linear models. 

Now we'll discuss a case study solution using multiple linear regression, and regularised linear regresison [Ridge and Lasso] . We'll also look at hyper parameter tuning for regularised regression.

A little background on the case study. This data belongs to a loan aggregator agency which connects loan applications to different financial institutions in attempt to get the best interest rate. They want to now utilise past data to predict interest rate given by any financial institute just by looking at loan application characteristics.

To achieve that , they have decided to do a POC with a data from a particular financial institution. The data is given in the file "loans data.csv". Lets begin: 

In [1]:
data_file=r'./Data/loans data.csv'
#data_file=C:/Users/YAMINI/Desktop/R fundamentals/Data/loans data.csv

import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
import numpy as np
from sklearn.model_selection import KFold
%matplotlib inline

ld=pd.read_csv(data_file)


In [2]:
ld.head()

,ID,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
0,81174.0,20000,20000,8.90%,36 months,debt_consolidation,14.90%,SC,MORTGAGE,6541.67,735-739,14,14272,2.0,< 1 year
1,99592.0,19200,19200,12.12%,36 months,debt_consolidation,28.36%,TX,MORTGAGE,4583.33,715-719,12,11140,1.0,2 years
2,80059.0,35000,35000,21.98%,60 months,debt_consolidation,23.81%,CA,MORTGAGE,11500.00,690-694,14,21977,1.0,2 years
3,15825.0,10000,9975,9.99%,36 months,debt_consolidation,14.30%,KS,MORTGAGE,3833.33,695-699,10,9346,0.0,5 years
4,33182.0,12000,12000,11.71%,36 months,credit_card,18.78%,NJ,RENT,3195.00,695-699,11,14469,0.0,9 years


In [72]:
ld.dtypes

ID                                float64
Amount.Requested                   object
Amount.Funded.By.Investors         object
Interest.Rate                      object
Loan.Length                        object
Loan.Purpose                       object
Debt.To.Income.Ratio               object
State                              object
Home.Ownership                     object
Monthly.Income                    float64
FICO.Range                         object
Open.CREDIT.Lines                  object
Revolving.CREDIT.Balance           object
Inquiries.in.the.Last.6.Months    float64
Employment.Length                  object
dtype: object

You can see that variable Interest.Rate and Debt.To.Income.Ratio contain "%" sign in their values and because of which they have come as character columns in the data. Lets remove these percentages first.

In [73]:
### The below code is simply removing the percentage sign from each column

for col in ["Interest.Rate","Debt.To.Income.Ratio"]:
    ld[col]=ld[col].astype("str")
    ld[col]=[x.replace("%","") for x in ld[col]]



In [74]:
ld.dtypes

ID                                float64
Amount.Requested                   object
Amount.Funded.By.Investors         object
Interest.Rate                      object
Loan.Length                        object
Loan.Purpose                       object
Debt.To.Income.Ratio               object
State                              object
Home.Ownership                     object
Monthly.Income                    float64
FICO.Range                         object
Open.CREDIT.Lines                  object
Revolving.CREDIT.Balance           object
Inquiries.in.the.Last.6.Months    float64
Employment.Length                  object
dtype: object

We can see that many columns which should have really been numbers have been imported as character columns , probably because some characters values in those columns in the files. We'll convert all such columns to numbers .

In [75]:
for col in ["Amount.Requested","Amount.Funded.By.Investors","Open.CREDIT.Lines","Revolving.CREDIT.Balance",
           "Inquiries.in.the.Last.6.Months","Interest.Rate","Debt.To.Income.Ratio"]:
    ld[col]=pd.to_numeric(ld[col],errors="coerce") ### errors = "coerce" will convert some value which are not numeric to NAN
#

In [76]:
ld.dtypes

ID                                float64
Amount.Requested                  float64
Amount.Funded.By.Investors        float64
Interest.Rate                     float64
Loan.Length                        object
Loan.Purpose                       object
Debt.To.Income.Ratio              float64
State                              object
Home.Ownership                     object
Monthly.Income                    float64
FICO.Range                         object
Open.CREDIT.Lines                 float64
Revolving.CREDIT.Balance          float64
Inquiries.in.the.Last.6.Months    float64
Employment.Length                  object
dtype: object

Next we will make dummy variables for remaining categorical variables

In [77]:
ld["Loan.Length"].value_counts()

36 months    1950
60 months     548
.               1
Name: Loan.Length, dtype: int64

Function get_dummies creates dummy variables for all the categories present in the categorical variable. Result is a dataframe, we can then choose and drop the dummies that we want to drop and attach the ones selected back to our original data.

In [78]:
ll_dummies=pd.get_dummies(ld["Loan.Length"])

In [79]:
ll_dummies.head()

,.,36 months,60 months
0,0,1,0
1,0,1,0
2,0,0,1
3,0,1,0
4,0,1,0


We'll add dummy variable for "36 months" to our data and ignore the rest two. 

In [80]:
ld["LL_36"]=ll_dummies["36 months"]

Now that we'are done with dataframe ll_dummies , we can drop it. Below we demonstrate a general way of removing variables from notebook environment.

In [81]:
%reset_selective ll_dummies

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


To know what all variables are in the environment, you can use function "who"

In [82]:
who

KFold	 Lasso	 LinearRegression	 Ridge	 col	 data_file	 i	 ld	 lp_dummies	 
math	 np	 pd	 train_test_split	 


Now that we have created dummies for Loan.Length, we need to remove this from the dataframe.

In [83]:
ld=ld.drop('Loan.Length',axis=1)

In [84]:
ld.dtypes

ID                                float64
Amount.Requested                  float64
Amount.Funded.By.Investors        float64
Interest.Rate                     float64
Loan.Purpose                       object
Debt.To.Income.Ratio              float64
State                              object
Home.Ownership                     object
Monthly.Income                    float64
FICO.Range                         object
Open.CREDIT.Lines                 float64
Revolving.CREDIT.Balance          float64
Inquiries.in.the.Last.6.Months    float64
Employment.Length                  object
LL_36                               uint8
dtype: object

Next we examine variable "Loan.Purpose".

In [85]:
ld["Loan.Purpose"].value_counts()

debt_consolidation    1307
credit_card            444
other                  200
home_improvement       152
major_purchase         101
small_business          87
car                     50
wedding                 39
medical                 30
moving                  29
vacation                21
house                   20
educational             15
renewable_energy         4
Name: Loan.Purpose, dtype: int64

There are 14 categories in that variable, we can either make 13 dummies or we can club few of these categories together and reduce the number of effective categories and then make dummy variables for those.

It makes sense to club those categories which behave similarly in terms of their effect on response. Or in other words , we can club those categories for which average interest rates are similar in the data.

In [86]:
print(round(ld.groupby("Loan.Purpose")["Interest.Rate"].mean()))
len(ld.index)

Loan.Purpose
car                   11.0
credit_card           13.0
debt_consolidation    14.0
educational           11.0
home_improvement      12.0
house                 13.0
major_purchase        11.0
medical               12.0
moving                14.0
other                 13.0
renewable_energy      10.0
small_business        13.0
vacation              12.0
wedding               12.0
Name: Interest.Rate, dtype: float64


2500

We can see from the table above that there are 4 effective categoris in the data. Lets club them

In [87]:
for i in range(len(ld.index)):
    if ld["Loan.Purpose"][i] in ["car","educational","major_purchase"]:
        ld.loc[i,"Loan.Purpose"]="cem"
    if ld["Loan.Purpose"][i] in ["home_improvement","medical","vacation","wedding"]:
        ld.loc[i,"Loan.Purpose"]="hmvw"
    if ld["Loan.Purpose"][i] in ["credit_card","house","other","small_business"]:
        ld.loc[i,"Loan.Purpose"]="chos"
    if ld["Loan.Purpose"][i] in ["debt_consolidation","moving"]:
        ld.loc[i,"Loan.Purpose"]="dm"

Now we make dummies for this variable

In [88]:
lp_dummies=pd.get_dummies(ld["Loan.Purpose"],prefix="LP")

In [89]:
lp_dummies.head()

,LP_cem,LP_chos,LP_dm,LP_hmvw,LP_renewable_energy
0,0,0,1,0,0
1,0,0,1,0,0
2,0,0,1,0,0
3,0,0,1,0,0
4,0,1,0,0,0


We'll add this data to original data. And then drop original variable "Loan.Purpose" and one of the dummies "LP_renewable_energy"

In [90]:
ld=pd.concat([ld,lp_dummies],1)
ld=ld.drop(["Loan.Purpose","LP_renewable_energy"],1)

In [91]:
ld.dtypes

ID                                float64
Amount.Requested                  float64
Amount.Funded.By.Investors        float64
Interest.Rate                     float64
Debt.To.Income.Ratio              float64
State                              object
Home.Ownership                     object
Monthly.Income                    float64
FICO.Range                         object
Open.CREDIT.Lines                 float64
Revolving.CREDIT.Balance          float64
Inquiries.in.the.Last.6.Months    float64
Employment.Length                  object
LL_36                               uint8
LP_cem                              uint8
LP_chos                             uint8
LP_dm                               uint8
LP_hmvw                             uint8
dtype: object

Next we look at variable "State".

In [92]:
ld["State"].nunique()

47

There are too many unique values. Although its not a legit reason to drop a variable. But we'll ignore this in this discussion any way in order to reduce amount of data prep that we are doing here. You can try including it in the model and see if the performance improves.

In [93]:
ld=ld.drop(["State"],1)

Next we take care of variable Home.Ownership.

In [94]:
ld["Home.Ownership"].value_counts()

MORTGAGE    1147
RENT        1146
OWN          200
OTHER          5
NONE           1
Name: Home.Ownership, dtype: int64

In [95]:
ld["ho_mort"]=np.where(ld["Home.Ownership"]=="MORTGAGE",1,0)
ld["ho_rent"]=np.where(ld["Home.Ownership"]=="RENT",1,0)
ld=ld.drop(["Home.Ownership"],1)

We have simply ignored values OTHER and NONE , and considered that there are only 3 categories and created only two dummies . We did this because of very low frequencies of OTHER and NONE

In [97]:
ld["FICO.Range"].head()

0    735-739
1    715-719
2    690-694
3    695-699
4    695-699
Name: FICO.Range, dtype: object

If you look at first few values of variable FICO.Range , you can see that we can convert it to numeric by taking average of the range given. To do that first we need to split the column with "-", so that we can have both end of ranges in separate columns and then we can simply average them.
Lets first split.

In [98]:
ld['f1'], ld['f2'] = zip(*ld['FICO.Range'].apply(lambda x: x.split('-', 1)))

Now we create new variable "fico" by averaging f1 and f2. And then we'll drop the original variable FICO.Range and f1,f2.

In [100]:
ld["fico"]=0.5*(pd.to_numeric(ld["f1"])+pd.to_numeric(ld["f2"]))

ld=ld.drop(["FICO.Range","f1","f2"],1)



KeyError: 'f1'

In [101]:
ld.head()

,ID,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Debt.To.Income.Ratio,Monthly.Income,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length,LL_36,LP_cem,LP_chos,LP_dm,LP_hmvw,ho_mort,ho_rent,fico
0,81174.0,20000.0,20000.0,8.90,14.90,6541.67,14.0,14272.0,2.0,< 1 year,1,0,0,1,0,1,0,737.0
1,99592.0,19200.0,19200.0,12.12,28.36,4583.33,12.0,11140.0,1.0,2 years,1,0,0,1,0,1,0,717.0
2,80059.0,35000.0,35000.0,21.98,23.81,11500.00,14.0,21977.0,1.0,2 years,0,0,0,1,0,1,0,692.0
3,15825.0,10000.0,9975.0,9.99,14.30,3833.33,10.0,9346.0,0.0,5 years,1,0,0,1,0,1,0,697.0
4,33182.0,12000.0,12000.0,11.71,18.78,3195.00,11.0,14469.0,0.0,9 years,1,0,1,0,0,0,1,697.0


Next we look at variavle Employment.Length. You'll see that we can convert that to number as well.

In [102]:
ld["Employment.Length"].value_counts()

10+ years    653
< 1 year     249
2 years      243
3 years      235
5 years      202
4 years      191
1 year       177
6 years      163
7 years      127
8 years      108
9 years       72
.              2
Name: Employment.Length, dtype: int64

In [103]:
ld["Employment.Length"]=ld["Employment.Length"].astype("str")
ld["Employment.Length"]=[x.replace("years","") for x in ld["Employment.Length"]]
ld["Employment.Length"]=[x.replace("year","") for x in ld["Employment.Length"]]

We can convert everything else to numbers , but "n/a" are a problem. We can look at average interest across all values of Employment.Length and then replace "n/a" with value which has closet average response.

In [104]:
round(ld.groupby("Employment.Length")["Interest.Rate"].mean(),2)

Employment.Length
.       11.34
1       12.49
10+     13.34
2       12.87
3       12.77
4       13.14
5       13.40
6       13.29
7       13.10
8       13.01
9       13.15
< 1     12.86
nan     12.78
Name: Interest.Rate, dtype: float64

As you can see "n/a" is similar to "< 1".

In [105]:
ld["Employment.Length"]=[x.replace("n/a","< 1") for x in ld["Employment.Length"]]
ld["Employment.Length"]=[x.replace("10+","10") for x in ld["Employment.Length"]]
ld["Employment.Length"]=[x.replace("< 1","0") for x in ld["Employment.Length"]]
ld["Employment.Length"]=pd.to_numeric(ld["Employment.Length"],errors="coerce")

In [106]:
ld.dtypes

ID                                float64
Amount.Requested                  float64
Amount.Funded.By.Investors        float64
Interest.Rate                     float64
Debt.To.Income.Ratio              float64
Monthly.Income                    float64
Open.CREDIT.Lines                 float64
Revolving.CREDIT.Balance          float64
Inquiries.in.the.Last.6.Months    float64
Employment.Length                 float64
LL_36                               uint8
LP_cem                              uint8
LP_chos                             uint8
LP_dm                               uint8
LP_hmvw                             uint8
ho_mort                             int32
ho_rent                             int32
fico                              float64
dtype: object

Now we have all the variables as numbers. After dropping observations with missing values , we can proceed to build oru model.

In [107]:
ld.shape

(2500, 18)

In [108]:
ld.dropna(axis=0,inplace=True)   #### This wil drop any record which has NA value in any one or more columns

In [109]:
ld.shape

(2394, 18)

In [110]:
ld.head()

,ID,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Debt.To.Income.Ratio,Monthly.Income,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length,LL_36,LP_cem,LP_chos,LP_dm,LP_hmvw,ho_mort,ho_rent,fico
0,81174.0,20000.0,20000.0,8.90,14.90,6541.67,14.0,14272.0,2.0,0.0,1,0,0,1,0,1,0,737.0
1,99592.0,19200.0,19200.0,12.12,28.36,4583.33,12.0,11140.0,1.0,2.0,1,0,0,1,0,1,0,717.0
2,80059.0,35000.0,35000.0,21.98,23.81,11500.00,14.0,21977.0,1.0,2.0,0,0,0,1,0,1,0,692.0
3,15825.0,10000.0,9975.0,9.99,14.30,3833.33,10.0,9346.0,0.0,5.0,1,0,0,1,0,1,0,697.0
4,33182.0,12000.0,12000.0,11.71,18.78,3195.00,11.0,14469.0,0.0,9.0,1,0,1,0,0,0,1,697.0


We now split our data into two random parts . One to build model on , Another to test its performance. Option "random_state" is used to make our random operation reproducible.

In [111]:
ld_train, ld_test = train_test_split(ld, test_size = 0.2,random_state=2) ### random_state is similar to set.seed(2S)

In [113]:
lm=LinearRegression()

Above line creates and object of class LinearRegression named lm. We can use this object to access all functions realted to LinearRegression.

Now we'll separate predictors and response for both the datasets . We'll also drop ID from predictor's list because it doesnt make sense to include an ID variable in the model. Variable "Amount.Funded.By.Investors" will also be dropped because it wont be available until the loan has been processed. We can use only those variables which are present at the point of the business process where we want to apply our model.

In [112]:
x_train=ld_train.drop(["Interest.Rate","ID","Amount.Funded.By.Investors"],1)
y_train=ld_train["Interest.Rate"]
x_test=ld_test.drop(["Interest.Rate","ID","Amount.Funded.By.Investors"],1)
y_test=ld_test["Interest.Rate"]

Now we can fit our model using lm the LinearRegression object that we created earlier

In [114]:
lm.fit(x_train,y_train) 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

Next we predict resposne on our test data , calculate errors on those prediction and then rmse for those residuals. That is the measure of performance on the test data. We can use this measure to compare other models that we'll build.

In [115]:
p_test=lm.predict(x_test)

residual=p_test-y_test

rmse_lm=np.sqrt(np.dot(residual,residual)/len(p_test))

rmse_lm

1.99319000650949

We can use this to compare our linear regression model with other techniques and evenutall pick the one with least error .

Next we show how to extract coefficient produced by our model

In [116]:
coefs=lm.coef_

features=x_train.columns

list(zip(features,coefs))

[('Amount.Requested', 0.0001520616264433721),
 ('Debt.To.Income.Ratio', 0.0014229632804925872),
 ('Monthly.Income', -1.471107262645891e-05),
 ('Open.CREDIT.Lines', -0.029040399129085133),
 ('Revolving.CREDIT.Balance', -4.061210679284056e-06),
 ('Inquiries.in.the.Last.6.Months', 0.362521331414258),
 ('Employment.Length', 0.0031443211037038296),
 ('LL_36', -3.229125497961786),
 ('LP_cem', -1.324387611792008),
 ('LP_chos', -1.4579324762102988),
 ('LP_dm', -1.515220381863035),
 ('LP_hmvw', -1.682759383322191),
 ('ho_mort', -0.4349433080678899),
 ('ho_rent', -0.15093058574020307),
 ('fico', -0.08657827156326188)]

We can see that linear regression has produced coefficients for all variables. If you recall our theoretical discussion, we need to penalise coefficient for the variables which are not really contributing well to our resposne and might be causing overfitting of the model. Among the regularised technique we'll first look at Ridge regression.

Since penalty in ridge regression is a hyperparameter , we'd look at multiple values of it and choose the best one through 10 fold cross validation.

In [121]:
# Finding best value of penalty weight with cross validation for ridge regression
alphas=np.linspace(2,10,100)
# We need to reset index for cross validation to work without hitch
x_train.reset_index(drop=True,inplace=True) ### re-assign index of rows to 0 to n again
y_train.reset_index(drop=True,inplace=True)
#x_train

In [122]:
alphas

array([ 2.        ,  2.08080808,  2.16161616,  2.24242424,  2.32323232,
        2.4040404 ,  2.48484848,  2.56565657,  2.64646465,  2.72727273,
        2.80808081,  2.88888889,  2.96969697,  3.05050505,  3.13131313,
        3.21212121,  3.29292929,  3.37373737,  3.45454545,  3.53535354,
        3.61616162,  3.6969697 ,  3.77777778,  3.85858586,  3.93939394,
        4.02020202,  4.1010101 ,  4.18181818,  4.26262626,  4.34343434,
        4.42424242,  4.50505051,  4.58585859,  4.66666667,  4.74747475,
        4.82828283,  4.90909091,  4.98989899,  5.07070707,  5.15151515,
        5.23232323,  5.31313131,  5.39393939,  5.47474747,  5.55555556,
        5.63636364,  5.71717172,  5.7979798 ,  5.87878788,  5.95959596,
        6.04040404,  6.12121212,  6.2020202 ,  6.28282828,  6.36363636,
        6.44444444,  6.52525253,  6.60606061,  6.68686869,  6.76767677,
        6.84848485,  6.92929293,  7.01010101,  7.09090909,  7.17171717,
        7.25252525,  7.33333333,  7.41414141,  7.49494949,  7.57

In [128]:
rmse_list=[]
for a in alphas:
    ridge = Ridge(fit_intercept=True, alpha=a)

    # computing average RMSE across 10-fold cross validation
   # kf = KFold(len(x_train), n_folds=10)
    kf =KFold(n_splits = 10)
    xval_err = 0
for train, test in kf.split(x_train):
    print(train)
    print(test)

[ 192  193  194 ... 1912 1913 1914]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191]
[   0    1    2 ... 1912 1913 1914]
[192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209
 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227
 2

In [129]:
rmse_list=[]
for a in alphas:
    ridge = Ridge(fit_intercept=True, alpha=a)

    # computing average RMSE across 10-fold cross validation
   # kf = KFold(len(x_train), n_folds=10)
    kf =KFold(n_splits = 10)
    xval_err = 0
    for train, test in kf.split(x_train):
        ridge.fit(x_train.loc[train], y_train[train])
        p = ridge.predict(x_train.loc[test])
        err = p - y_train[test]
        xval_err += np.dot(err,err)
    rmse_10cv = np.sqrt(xval_err/len(x_train))
    # uncomment below to print rmse values for individidual alphas
#     print('{:.3f}\t {:.6f}\t '.format(a,rmse_10cv))
    rmse_list.extend([rmse_10cv])
best_alpha=alphas[rmse_list==min(rmse_list)]
print('Alpha with min 10cv error is : ',best_alpha )

Alpha with min 10cv error is :  [2.]


best value of alpha  might be slightly different across different runs because of random nature of cross validation. So dont worry if you determine a different value of best alpha.

Next we fit Ridge Regression on the entire train data with best value of alpha we just determined.


In [130]:

ridge=Ridge(fit_intercept=True,alpha=best_alpha)

ridge.fit(x_train,y_train)

p_test=ridge.predict(x_test)

residual=p_test-y_test

rmse_ridge=np.sqrt(np.dot(residual,residual)/len(p_test))

rmse_ridge

1.9880860963814568

In [131]:
list(zip(x_train.columns,ridge.coef_))

[('Amount.Requested', 0.00015251694372239017),
 ('Debt.To.Income.Ratio', 0.0014775097004255415),
 ('Monthly.Income', -1.5128805942744445e-05),
 ('Open.CREDIT.Lines', -0.029628866307200446),
 ('Revolving.CREDIT.Balance', -4.063218246743193e-06),
 ('Inquiries.in.the.Last.6.Months', 0.36242046047679877),
 ('Employment.Length', 0.003179402831006199),
 ('LL_36', -3.203694777132437),
 ('LP_cem', -0.23155548627444542),
 ('LP_chos', -0.3701505663485961),
 ('LP_dm', -0.42718370127604405),
 ('LP_hmvw', -0.5887684435728163),
 ('ho_mort', -0.42133963041234235),
 ('ho_rent', -0.14094866291933478),
 ('fico', -0.08656728469240173)]

In [132]:
list(zip(features, coefs))

[('Amount.Requested', 0.0001520616264433721),
 ('Debt.To.Income.Ratio', 0.0014229632804925872),
 ('Monthly.Income', -1.471107262645891e-05),
 ('Open.CREDIT.Lines', -0.029040399129085133),
 ('Revolving.CREDIT.Balance', -4.061210679284056e-06),
 ('Inquiries.in.the.Last.6.Months', 0.362521331414258),
 ('Employment.Length', 0.0031443211037038296),
 ('LL_36', -3.229125497961786),
 ('LP_cem', -1.324387611792008),
 ('LP_chos', -1.4579324762102988),
 ('LP_dm', -1.515220381863035),
 ('LP_hmvw', -1.682759383322191),
 ('ho_mort', -0.4349433080678899),
 ('ho_rent', -0.15093058574020307),
 ('fico', -0.08657827156326188)]

You can see that ridge regression though, shrinks the coefficients but never makes them exactly zero, essentially never reduce our model size. Next we look at lasso Regression.

In [136]:
alphas=np.linspace(0.0001,1,100)
rmse_list=[]
for a in alphas:
    lasso = Lasso(fit_intercept=True, alpha=a,max_iter=10000)

    # computing RMSE using 10-fold cross validation
    kf = KFold(n_splits=10)
    xval_err = 0
    for train, test in kf.split(x_train):
        lasso.fit(x_train.loc[train], y_train[train])
        p =lasso.predict(x_train.loc[test])
        err = p - y_train[test]
        xval_err += np.dot(err,err)
    rmse_10cv = np.sqrt(xval_err/len(x_train))
    rmse_list.extend([rmse_10cv])
    # Uncomment below to print rmse values of individual alphas
    print('{:.3f}\t {:.4f}\t '.format(a,rmse_10cv))
best_alpha=alphas[rmse_list==min(rmse_list)]
print('Alpha with min 10cv error is : ',best_alpha )

0.000	 2.0745	 
0.010	 2.0741	 
0.020	 2.0737	 
0.030	 2.0752	 
0.041	 2.0774	 
0.051	 2.0803	 
0.061	 2.0834	 
0.071	 2.0861	 
0.081	 2.0888	 
0.091	 2.0919	 
0.101	 2.0953	 
0.111	 2.0992	 
0.121	 2.1035	 
0.131	 2.1081	 
0.141	 2.1131	 
0.152	 2.1184	 
0.162	 2.1242	 
0.172	 2.1302	 
0.182	 2.1367	 
0.192	 2.1434	 
0.202	 2.1505	 
0.212	 2.1580	 
0.222	 2.1657	 
0.232	 2.1739	 
0.242	 2.1823	 
0.253	 2.1911	 
0.263	 2.2002	 
0.273	 2.2096	 
0.283	 2.2193	 
0.293	 2.2293	 
0.303	 2.2396	 
0.313	 2.2503	 
0.323	 2.2612	 
0.333	 2.2724	 
0.343	 2.2839	 
0.354	 2.2957	 
0.364	 2.3078	 
0.374	 2.3201	 
0.384	 2.3327	 
0.394	 2.3456	 
0.404	 2.3588	 
0.414	 2.3722	 
0.424	 2.3858	 
0.434	 2.3997	 
0.444	 2.4138	 
0.455	 2.4272	 
0.465	 2.4369	 
0.475	 2.4402	 
0.485	 2.4432	 
0.495	 2.4447	 
0.505	 2.4462	 
0.515	 2.4478	 
0.525	 2.4493	 
0.535	 2.4507	 
0.545	 2.4519	 
0.556	 2.4528	 
0.566	 2.4533	 
0.576	 2.4535	 
0.586	 2.4535	 
0.596	 2.4535	 
0.606	 2.4535	 
0.616	 2.4535	 
0.626	 2

In [137]:
lasso=Lasso(fit_intercept=True,alpha=best_alpha)

lasso.fit(x_train,y_train)

p_test=lasso.predict(x_test)

residual=p_test-y_test

rmse_lasso=np.sqrt(np.dot(residual,residual)/len(p_test))

rmse_lasso

1.982894911099847

In [138]:
list(zip(x_train.columns,lasso.coef_))

[('Amount.Requested', 0.00015392932840652785),
 ('Debt.To.Income.Ratio', 0.0009321941705340002),
 ('Monthly.Income', -1.7893764707713713e-05),
 ('Open.CREDIT.Lines', -0.029156552255009734),
 ('Revolving.CREDIT.Balance', -4.210314897401005e-06),
 ('Inquiries.in.the.Last.6.Months', 0.34610251064075437),
 ('Employment.Length', 0.0),
 ('LL_36', -3.0814760599035815),
 ('LP_cem', 0.0),
 ('LP_chos', 0.0),
 ('LP_dm', -0.0),
 ('LP_hmvw', -0.0),
 ('ho_mort', -0.21660653356105106),
 ('ho_rent', 0.0),
 ('fico', -0.0866913912899347)]

We can see that lasso regression, not only improves performance on the data slightly , but also makes size of the model smaller by making many coefficents exactly zero, thus excluding them from our model.

### Logistic Model for Binary Classification

A retail banking institution is going to float a stock trading facility for their existing customer. Since this kind of facitlity is nothing new , company knows that they will have to incetivise their customers for adopting their offerings. One way to incetiwise is to offer discounts on the commision for trading transactions.

One issue with that is that only about 10% of the customers do enought trades for earnings after discounts to be profitable. Company wants to figure out, which are those 10% customer so that it can selectively offer them discount. there is no magic way to figure that out. So company rolled out this service to about 10000+ of their customers and observed their trading behaviour for 6 months and after that they labelled them into two revenue.grids 1 and 2.
using this data, now they want us to build a classification model which can be used to classify their remaining customers into these revenue grids.



#### Logistic Regression from Scikit Learn 

Logistic Regression in scikit learn already contains penalties. l1 and l2 [Read as L-one & L-two] penalties . l1 penalty is same as lasso penalty where as l2 is same as ridge penalty. parameter C for logistic regression function is the hyperparameter for penalty . However it works in inverse fashion, i.e. if C takes smalle , it means higher penalty. 

For the case that we have discussed here , we have discussed l1 penalty with value of C as 1. We have left following things for you to try on your own.

* model with l2 penalty
* Finding optimal value of hyperparameter C with cross-validation for both the penalties

You will find these in the practice exercise as well

you can use auc value obtained from function roc_auc_score to select best value for the hyperparameter. Higher the auc, better is the model . If you dont recall this, please go back to the theoretical reading material.

Lets beging our model building process.


In [139]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [140]:
data_file=r'./Data/Existing Base.csv'
bd=pd.read_csv(data_file)

In [141]:
bd.head()

,REF_NO,children,age_band,status,occupation,occupation_partner,home_status,family_income,self_employed,self_employed_partner,...,Investment Tax Saving Bond,Home Loan,Online Purchase Amount,Revenue Grid,gender,region,Investment in Commudity,Investment in Equity,Investment in Derivative,Portfolio Balance
0,1,Zero,51-55,Partner,Manual Worker,Secretarial/Admin,Own Home,"<17,500, >=15,000",No,No,...,19.99,0.00,0.00,1,Female,Wales,74.67,18.66,32.32,89.43
1,2,Zero,55-60,Single/Never Married,Retired,Retired,Own Home,"<27,500, >=25,000",No,No,...,0.00,0.00,0.00,2,Female,North West,20.19,0.00,4.33,22.78
2,3,Zero,26-30,Single/Never Married,Professional,Other,Own Home,"<30,000, >=27,500",Yes,No,...,0.00,3.49,0.00,2,Male,North,98.06,31.07,80.96,171.78
3,5,Zero,18-21,Single/Never Married,Professional,Manual Worker,Own Home,"<15,000, >=12,500",No,No,...,0.00,0.00,0.00,2,Female,West Midlands,4.10,14.15,17.57,-41.70
4,6,Zero,45-50,Partner,Business Manager,Unknown,Own Home,"<30,000, >=27,500",No,No,...,0.00,45.91,25.98,2,Female,Scotland,70.16,55.86,80.44,235.02


In [142]:
bd["children"].value_counts()

Zero    6208
1       1848
2       1607
3        473
4+        19
Name: children, dtype: int64

It seems we can directly convert this to numeric.

In [144]:
bd.loc[bd["children"]=="Zero","children"]="0"
bd.loc[bd["children"]=="4+","children"]="4"
bd["children"]=pd.to_numeric(bd["children"],errors="coerce")  #### If after the above two codes, you 

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [145]:
bd["Revenue Grid"].value_counts()

2    9069
1    1086
Name: Revenue Grid, dtype: int64

In [146]:
bd["y"]=np.where(bd["Revenue Grid"]==2,0,1)
bd=bd.drop(["Revenue Grid"],1)

For variable , age_band if we treat it as categorical variable , we can combine its categories by looking average response rate across its categories.

In [147]:
round(bd.groupby("age_band")["y"].mean(),2)

age_band
18-21      0.17
22-25      0.11
26-30      0.11
31-35      0.11
36-40      0.13
41-45      0.11
45-50      0.10
51-55      0.10
55-60      0.11
61-65      0.09
65-70      0.10
71+        0.10
Unknown    0.05
Name: y, dtype: float64

In [148]:
for i in range(len(bd)):
    if bd["age_band"][i] in ["71+","65-70","51-55","45-50"]:
        bd.loc[i,"age_band"]="ab_10"
    if bd["age_band"][i] in ["55-60","41-45","31-35","22-25","26-30"]:
        bd.loc[i,"age_band"]="ab_11"
    if bd["age_band"][i]=="36-40":
        bd.loc[i,"age_band"]="ab_13"
    if bd["age_band"][i]=="18-21":
        bd.loc[i,"age_band"]="ab_17"
    if bd["age_band"][i]=="61-65":
        bd.loc[i,"age_band"]="ab_9"
ab_dummies=pd.get_dummies(bd["age_band"])
ab_dummies.head()

,Unknown,ab_10,ab_11,ab_13,ab_17,ab_9
0,0,1,0,0,0,0
1,0,0,1,0,0,0
2,0,0,1,0,0,0
3,0,0,0,0,1,0
4,0,1,0,0,0,0


We will add it back to the dataset, dropping the dummy for "Unknown".

In [149]:
bd=pd.concat([bd,ab_dummies],1)
bd=bd.drop(["age_band","Unknown"],1)

In [174]:
bd["status"].value_counts()

Partner                 7709
Single/Never Married    1101
Divorced/Separated       679
Widowed                  618
Unknown                   48
Name: status, dtype: int64

In [175]:
bd["st_partner"]=np.where(bd["status"]=="Partner",1,0)
bd["st_singleNm"]=np.where(bd["status"]=="Single/Never Married",1,0)
bd["st_divSep"]=np.where(bd["status"]=="Divorced/Separated",1,0)
bd=bd.drop(["status"],1)

In [151]:
round(bd.groupby("occupation")["y"].mean(),2)

occupation
Business Manager     0.12
Housewife            0.09
Manual Worker        0.11
Other                0.11
Professional         0.12
Retired              0.10
Secretarial/Admin    0.11
Student              0.11
Unknown              0.11
Name: y, dtype: float64

In [152]:
for i in range(len(bd)):
    if bd["occupation"][i] in ["Unknown","Student","Secretarial/Admin","Other","Manual Worker"]:
        bd.loc[i,"occupation"]="oc_11"
    if bd["occupation"][i] in ["Professional","Business Manager"]:
        bd.loc[i,"occupation"]="oc_12"
    if bd["occupation"][i]=="Retired":
        bd.loc[i,"occupation"]="oc_10"
oc_dummies=pd.get_dummies(bd["occupation"])
oc_dummies.head()

,Housewife,oc_10,oc_11,oc_12
0,0,0,1,0
1,0,1,0,0
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1


In [155]:
bd=pd.concat([bd,oc_dummies],1)

bd=bd.drop(["occupation","Housewife"],1)

KeyError: "['occupation' 'Housewife'] not found in axis"

In [156]:
round(bd.groupby("occupation_partner")["y"].mean(),2)

occupation_partner
Business Manager     0.11
Housewife            0.11
Manual Worker        0.11
Other                0.10
Professional         0.11
Retired              0.10
Secretarial/Admin    0.12
Student              0.12
Unknown              0.10
Name: y, dtype: float64

In [157]:
bd["ocp_10"]=0
bd["ocp_12"]=0
for i in range(len(bd)):
    if bd["occupation_partner"][i] in ["Unknown","Retired","Other"]:
        bd.loc[i,"ocp_10"]=1
    if bd["occupation_partner"][i] in ["Student","Secretarial/Admin"]:
        bd.loc[i,"ocp_12"]=1
        
bd=bd.drop(["occupation_partner","TVarea","post_code","post_area","region"],1)

You can see that we have also dropped variables TVarea, region, post_code, post_area. If you look at number of unique values taken by post_area and post_code , you'll realise why decided to drop them. TVarea and region on the other hand we have left for you to make use of and see if using them improves your model.

In [158]:
bd["home_status"].value_counts()

Own Home                9413
Rent from Council/HA     322
Rent Privately           261
Live in Parental Hom     109
Unclassified              50
Name: home_status, dtype: int64

In [176]:
bd["hs_own"]=np.where(bd["home_status"]=="Own Home",1,0)
del bd["home_status"]

Notice that we used an alternate syntax for dropping a column here. You can use that too if you like this syntax better.

In [161]:
bd["gender"].value_counts()

KeyError: 'gender'

In [160]:
bd["gender_f"]=np.where(bd["gender"]=="Female",1,0)
del bd["gender"]

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [162]:
bd.head()

,REF_NO,children,status,home_status,family_income,self_employed,self_employed_partner,year_last_moved,Average Credit Card Transaction,Balance Transfer,...,oc_10,oc_11,oc_12,Housewife,oc_10,oc_11,oc_12,ocp_10,ocp_12,gender_f
0,1,0,Partner,Own Home,"<17,500, >=15,000",No,No,1972,148.44,142.95,...,0,1,0,0,0,1,0,0,1,1
1,2,0,Single/Never Married,Own Home,"<27,500, >=25,000",No,No,1998,0.00,74.98,...,1,0,0,0,1,0,0,1,0,1
2,3,0,Single/Never Married,Own Home,"<30,000, >=27,500",Yes,No,1996,0.00,166.44,...,0,0,1,0,0,0,1,1,0,0
3,5,0,Single/Never Married,Own Home,"<15,000, >=12,500",No,No,1997,0.00,0.00,...,0,0,1,0,0,0,1,0,0,1
4,6,0,Partner,Own Home,"<30,000, >=27,500",No,No,1995,73.45,57.96,...,0,0,1,0,0,0,1,1,0,1


In [163]:
bd["self_employed"].value_counts()

No     9436
Yes     719
Name: self_employed, dtype: int64

In [164]:
bd["semp_yes"]=np.where(bd["self_employed"]=="Yes",1,0)
del bd["self_employed"]

In [165]:
bd["self_employed_partner"].value_counts()

No     9026
Yes    1129
Name: self_employed_partner, dtype: int64

In [166]:
bd["semp_part_yes"]=np.where(bd["self_employed_partner"]=="Yes",1,0)
del bd["self_employed_partner"]

In [75]:
bd["family_income"].value_counts()

>=35,000             2517
<27,500, >=25,000    1227
<30,000, >=27,500     994
<25,000, >=22,500     833
<20,000, >=17,500     683
<12,500, >=10,000     677
<17,500, >=15,000     634
<15,000, >=12,500     629
<22,500, >=20,000     590
<10,000, >= 8,000     563
< 8,000, >= 4,000     402
< 4,000               278
Unknown               128
Name: family_income, dtype: int64

We can convert this to number as average of the range once we have figured out what to do with category "Unknown".

In [76]:
round(bd.groupby("family_income")["y"].mean(),4)

family_income
< 4,000              0.0755
< 8,000, >= 4,000    0.0796
<10,000, >= 8,000    0.1066
<12,500, >=10,000    0.1019
<15,000, >=12,500    0.1113
<17,500, >=15,000    0.1230
<20,000, >=17,500    0.1113
<22,500, >=20,000    0.1186
<25,000, >=22,500    0.1032
<27,500, >=25,000    0.0970
<30,000, >=27,500    0.1157
>=35,000             0.1116
Unknown              0.0703
Name: y, dtype: float64

In [168]:
bd["fi"]=4 # by doing this , we have essentially clubbed <4000 and Unknown values . How?
bd.loc[bd["family_income"]=="< 8,000, >= 4,000","fi"]=6
bd.loc[bd["family_income"]=="<10,000, >= 8,000","fi"]=9
bd.loc[bd["family_income"]=="<12,500, >=10,000","fi"]=11.25
bd.loc[bd["family_income"]=="<15,000, >=12,500","fi"]=13.75
bd.loc[bd["family_income"]=="<17,500, >=15,000","fi"]=16.25
bd.loc[bd["family_income"]=="<20,000, >=17,500","fi"]=18.75
bd.loc[bd["family_income"]=="<22,500, >=20,000","fi"]=21.25
bd.loc[bd["family_income"]=="<25,000, >=22,500","fi"]=23.75
bd.loc[bd["family_income"]=="<27,500, >=25,000","fi"]=26.25
bd.loc[bd["family_income"]=="<30,000, >=27,500","fi"]=28.75
bd.loc[bd["family_income"]==">=35,000","fi"]=35
bd=bd.drop(["family_income"],1)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


KeyError: 'family_income'

In [181]:
bd.dtypes

REF_NO                               int64
children                             int64
year_last_moved                      int64
Average Credit Card Transaction    float64
Balance Transfer                   float64
Term Deposit                       float64
Life Insurance                     float64
Medical Insurance                  float64
Average A/C Balance                float64
Personal Loan                      float64
Investment in Mutual Fund          float64
Investment Tax Saving Bond         float64
Home Loan                          float64
Online Purchase Amount             float64
Investment in Commudity            float64
Investment in Equity               float64
Investment in Derivative           float64
Portfolio Balance                  float64
y                                    int32
ab_10                                uint8
ab_11                                uint8
ab_13                                uint8
ab_17                                uint8
ab_9       

Now that the entire data is of numeric type, lets beging our modelling process after removing nas from the data.

In [185]:
bd.dropna(axis=0,inplace=True)
bd_train, bd_test = train_test_split(bd, test_size = 0.2,random_state=2)

In [186]:
x_train=bd_train.drop(["y","REF_NO"],1)
y_train=bd_train["y"]
x_test=bd_test.drop(["y","REF_NO"],1)
y_test=bd_test["y"]

In [187]:
logr=LogisticRegression(penalty="l1",class_weight="balanced",random_state=2)

In [188]:
logr.fit(x_train,y_train)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l1', random_state=2,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [189]:
# score model performance on the test data
roc_auc_score(y_test,logr.predict(x_test))

0.8998685666851134

To arrive at the eventual 1,0 prediction , we need to find some way [some cutoff ] to convert predicted probabilites into two classes. Lets first get the probabilities out.

In [191]:
prob_score=pd.Series(list(zip(*logr.predict_proba(x_train)))[1]) ### This * will help you convert each of total rows in dataset 
#into a tuple.
prob_score

0       0.038004
1       0.038500
2       0.014712
3       0.246622
4       0.037817
5       0.181598
6       0.027525
7       0.020924
8       0.003909
9       1.000000
10      0.175340
11      0.021994
12      0.018171
13      0.030818
14      0.022829
15      0.001234
16      0.000071
17      0.144349
18      0.019179
19      0.048694
20      0.000957
21      0.292059
22      0.673183
23      0.019348
24      0.043944
25      0.040277
26      0.153029
27      0.933968
28      0.050677
29      0.000237
          ...   
8094    0.058734
8095    0.081181
8096    0.993924
8097    0.974277
8098    0.999791
8099    0.038730
8100    0.202923
8101    0.071032
8102    0.999999
8103    0.106071
8104    0.226547
8105    0.093667
8106    0.074512
8107    0.249256
8108    0.999997
8109    0.074780
8110    0.153765
8111    0.070772
8112    0.041469
8113    0.007109
8114    0.235128
8115    0.121223
8116    0.412382
8117    0.002785
8118    0.117806
8119    0.005984
8120    0.002901
8121    0.1378

On these scores , we will consider many cutoffs between 0 to 1

In [212]:
cutoffs=np.linspace(0,1,100)
cutoffs

array([0.        , 0.01010101, 0.02020202, 0.03030303, 0.04040404,
       0.05050505, 0.06060606, 0.07070707, 0.08080808, 0.09090909,
       0.1010101 , 0.11111111, 0.12121212, 0.13131313, 0.14141414,
       0.15151515, 0.16161616, 0.17171717, 0.18181818, 0.19191919,
       0.2020202 , 0.21212121, 0.22222222, 0.23232323, 0.24242424,
       0.25252525, 0.26262626, 0.27272727, 0.28282828, 0.29292929,
       0.3030303 , 0.31313131, 0.32323232, 0.33333333, 0.34343434,
       0.35353535, 0.36363636, 0.37373737, 0.38383838, 0.39393939,
       0.4040404 , 0.41414141, 0.42424242, 0.43434343, 0.44444444,
       0.45454545, 0.46464646, 0.47474747, 0.48484848, 0.49494949,
       0.50505051, 0.51515152, 0.52525253, 0.53535354, 0.54545455,
       0.55555556, 0.56565657, 0.57575758, 0.58585859, 0.5959596 ,
       0.60606061, 0.61616162, 0.62626263, 0.63636364, 0.64646465,
       0.65656566, 0.66666667, 0.67676768, 0.68686869, 0.6969697 ,
       0.70707071, 0.71717172, 0.72727273, 0.73737374, 0.74747

For each of these cutoff , we are going to look at TP,FP,TN,FN values and caluclate KS. Then we'll chose the best cutoff as the one having highest KS.

In [195]:
predicted=pd.Series([0]*len(y_train))
predicted

0       0
1       0
2       0
3       0
4       0
5       0
6       0
7       0
8       0
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      0
24      0
25      0
26      0
27      0
28      0
29      0
       ..
8094    0
8095    0
8096    0
8097    0
8098    0
8099    0
8100    0
8101    0
8102    0
8103    0
8104    0
8105    0
8106    0
8107    0
8108    0
8109    0
8110    0
8111    0
8112    0
8113    0
8114    0
8115    0
8116    0
8117    0
8118    0
8119    0
8120    0
8121    0
8122    0
8123    0
Length: 8124, dtype: int64

In [213]:
predicted[prob_score>cutoff]

9       1
34      1
35      1
79      1
104     1
125     1
173     1
190     1
220     1
257     1
276     1
277     1
300     1
302     1
306     1
364     1
406     1
489     1
508     1
525     1
529     1
558     1
563     1
576     1
600     1
631     1
633     1
639     1
651     1
663     1
       ..
7638    1
7653    1
7654    1
7669    1
7686    1
7711    1
7718    1
7738    1
7751    1
7754    1
7777    1
7788    1
7803    1
7814    1
7826    1
7846    1
7850    1
7878    1
7906    1
7959    1
8002    1
8018    1
8032    1
8084    1
8089    1
8096    1
8098    1
8102    1
8108    1
8123    1
Length: 361, dtype: int64

In [210]:
KS_cut=[]
for cutoff in cutoffs:
    predicted=pd.Series([0]*len(y_train))
    predicted[prob_score>cutoff]=1
    df=pd.DataFrame(list(zip(y_train,predicted)),columns=["real","predicted"])
    TP=len(df[(df["real"]==1) &(df["predicted"]==1) ])
    FP=len(df[(df["real"]==0) &(df["predicted"]==1) ])
    TN=len(df[(df["real"]==0) &(df["predicted"]==0) ])
    FN=len(df[(df["real"]==1) &(df["predicted"]==0) ])
    P=TP+FN
    N=TN+FP
    KS=(TP/P)-(FP/N)
    KS_cut.append(KS)

cutoff_data=pd.DataFrame(list(zip(cutoffs,KS_cut)),columns=["cutoff","KS"])

KS_cutoff=cutoff_data[cutoff_data["KS"]==cutoff_data["KS"].max()]["cutoff"]


### F1 score: find out and get maximum F1 score 
## F1 = (2* Precision* Recall)/(Precision + Recall)
prob_score

0       0.038004
1       0.038500
2       0.014712
3       0.246622
4       0.037817
5       0.181598
6       0.027525
7       0.020924
8       0.003909
9       1.000000
10      0.175340
11      0.021994
12      0.018171
13      0.030818
14      0.022829
15      0.001234
16      0.000071
17      0.144349
18      0.019179
19      0.048694
20      0.000957
21      0.292059
22      0.673183
23      0.019348
24      0.043944
25      0.040277
26      0.153029
27      0.933968
28      0.050677
29      0.000237
          ...   
8094    0.058734
8095    0.081181
8096    0.993924
8097    0.974277
8098    0.999791
8099    0.038730
8100    0.202923
8101    0.071032
8102    0.999999
8103    0.106071
8104    0.226547
8105    0.093667
8106    0.074512
8107    0.249256
8108    0.999997
8109    0.074780
8110    0.153765
8111    0.070772
8112    0.041469
8113    0.007109
8114    0.235128
8115    0.121223
8116    0.412382
8117    0.002785
8118    0.117806
8119    0.005984
8120    0.002901
8121    0.1378

Now we'll see how this model with the cutoff determined here , performs on the test data.

In [203]:
# Performance on test data

prob_score=pd.Series(list(zip(*logr.predict_proba(x_train)))[1])
prob_score_test=pd.Series(list(zip(*logr.predict_proba(x_test)))[1])

predicted_test=pd.Series([0]*len(y_test))
predicted_test[prob_score_test>float(KS_cutoff)]=1

df_test=pd.DataFrame(list(zip(y_test,predicted_test)),columns=["real","predicted"])

k=pd.crosstab(df_test['real'],df_test["predicted"])
print('confusion matrix :\n \n ',k)
TN=k.iloc[0,0]
TP=k.iloc[1,1]
FP=k.iloc[0,1]
FN=k.iloc[1,0]
P=TP+FN
N=TN+FP

### From sklearn.metrics import confusion matrix
## confusion_matrix(df_test['real'], df_test['predicted'])

confusion matrix :
 
  predicted     0    1
real                
0          1647  160
1            26  198


In [204]:
# Accuracy of test
(TP+TN)/(P+N)

0.9084194977843427

In [206]:
# Sensitivity on test
TP/P

0.8839285714285714

In [207]:
#Specificity on test
TN/N

0.9114554510237963

Next we see how cutoff determined by F_beta score performs on test data for beta values : 0.5,1,2

In [208]:
cutoffs=np.linspace(0.010,0.99,100)
def Fbeta_perf(beta,cutoffs,y_train,prob_score):
    FB_cut=[]
    for cutoff in cutoffs:
        predicted=pd.Series([0]*len(y_train))
        predicted[prob_score>cutoff]=1
        df=pd.DataFrame(list(zip(y_train,predicted)),columns=["real","predicted"])

        TP=len(df[(df["real"]==1) &(df["predicted"]==1) ])
        FP=len(df[(df["real"]==0) &(df["predicted"]==1) ])
        FN=len(df[(df["real"]==1) &(df["predicted"]==0) ])
        P=TP+FN
        
        
        Precision=TP/(TP+FP)
        Recall=TP/P
        FB=(1+beta**2)*Precision*Recall/((beta**2)*Precision+Recall)
        FB_cut.append(FB)

    cutoff_data=pd.DataFrame(list(zip(cutoffs,FB_cut)),columns=["cutoff","FB"])

    FB_cutoff=cutoff_data[cutoff_data["FB"]==cutoff_data["FB"].max()]["cutoff"]

    prob_score_test=pd.Series(list(zip(*logr.predict_proba(x_test)))[1])

    predicted_test=pd.Series([0]*len(y_test))
    predicted_test[prob_score_test>float(FB_cutoff)]=1

    df_test=pd.DataFrame(list(zip(y_test,predicted_test)),columns=["real","predicted"])

    k=pd.crosstab(df_test['real'],df_test["predicted"])
#     print('confusion matrix :\n \n ',k)
    TN=k.iloc[0,0]
    TP=k.iloc[1,1]
    FP=k.iloc[0,1]
    FN=k.iloc[1,0]
    P=TP+FN
    N=TN+FP
    print('For beta :',beta)
    print('Accuracy is :',(TP+TN)/(P+N))
    print('Sensitivity is :',(TP/P))
    print('Specificity is :',(TN/N))
    print('\n \n \n')

In [209]:
Fbeta_perf(0.5,cutoffs,y_train,prob_score)
Fbeta_perf(1,cutoffs,y_train,prob_score)
Fbeta_perf(2,cutoffs,y_train,prob_score)

For beta : 0.5
Accuracy is : 0.9389463318562284
Sensitivity is : 0.6071428571428571
Specificity is : 0.9800774764803541

 
 

For beta : 1
Accuracy is : 0.9325455440669621
Sensitivity is : 0.7678571428571429
Specificity is : 0.9529607083563918

 
 

For beta : 2
Accuracy is : 0.9281142294436239
Sensitivity is : 0.8392857142857143
Specificity is : 0.93912562257886

 
 



You can see that low beta < 1 favors Specificity where as beta > 1 favors sensitivity

We'll conclude our discussion here. Please do the practice exercises . If you face any issue we'll discuss that either in class or QA forum on LMS.

Prepared By : Lalit Sachan (lalit.sachan@edvancer.in)

In case of any doubts or errata alert; please take to QA forum for discussion.

Doubts will be discussed in live class sessions too. [This doesnt apply for self paced students]